<a href="https://colab.research.google.com/github/soumyashubham10/IEEE-ML/blob/main/Soumya_Shubham_IEEE_ML_Round_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 42.6 MB/s eta 0:00:00


 Environment Setup and Library Installation


In [2]:
from datasets import load_dataset

dataset = load_dataset(
    "ms_marco",
    "v1.1",
    split="train[:1%]"   # ~90k passages
)

print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

v1.1/validation-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

v1.1/train-00000-of-00001.parquet:   0%|          | 0.00/175M [00:00<?, ?B/s]

v1.1/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 823
})


This cell loads a small subset of the MS MARCO v1.1 dataset using the HuggingFace datasets library.
Only 1% of the training split is loaded to ensure fast execution in Google Colab.

Hugging Face is used because it provides easy, reliable models. It allows direct loading of datasets like MS MARCO without manual downloads.



In [3]:
print(dataset.column_names)


['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers']


This command displays the available columns in the MS MARCO dataset.
It helps to understand the dataset structure before processing

In [4]:
{
  'passage_text': '...',
  'is_selected': 1
}


{'passage_text': '...', 'is_selected': 1}

In [5]:
dataset = load_dataset(
    "ms_marco",
    "v1.1",
    split="train[:10%]"
)

print("Queries loaded:", len(dataset))


Queries loaded: 8233


This code loads a portion of the MS MARCO dataset using the Hugging Face load_dataset function. Only 10% of the training data is loaded, which helps reduce memory usage and makes experimentation faster. The len(dataset) statement then prints the total number of query samples that were successfully loaded.

In [6]:
queries = []
relevant_docs = []

for item in dataset:
    queries.append(item["query"])

    rel = set()
    for text, sel in zip(item["passages"]["passage_text"],
                         item["passages"]["is_selected"]):
        if sel == 1:
            rel.add(text)

    relevant_docs.append(rel)

print("Total queries:", len(queries))


Total queries: 8233


In [7]:
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

This step imports some standard libraries.

In [8]:
import random
import numpy as np
import torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


This code is used to fix the randomness in a program.
The same seed value (42) is given to random, NumPy, and PyTorch so that every time the code runs, it produces the same random results.
This helps in reproducibility, meaning experiments and results can be repeated and verified easil

In [9]:
texts = []
doc_ids = []

for item in dataset:
    for passage in item["passages"]["passage_text"]:
        texts.append(passage)
        doc_ids.append(len(doc_ids))
        if len(texts) >= 75000:
            break
    if len(texts) >= 75000:
        break

corpus_df = pd.DataFrame({
    "doc_id": doc_ids,
    "text": texts
})

print("Total passages extracted:", len(corpus_df))
corpus_df.head()


Total passages extracted: 67656


,doc_id,text
0,0,"Since 2007, the RBA's outstanding reputation h..."
1,1,The Reserve Bank of Australia (RBA) came into ...
2,2,RBA Recognized with the 2014 Microsoft US Regi...
3,3,The inner workings of a rebuildable atomizer a...
4,4,Results-Based Accountability® (also known as R...


This cell extracts individual passages from the MS MARCO dataset and builds a document corpus.Each query in MS MARCO contains multiple passages.Passages are extracted sequentially and stored in a list.
A unique doc_id is assigned to each passage.
This DataFrame serves as the database for the semantic search engine.


In [10]:
df = pd.DataFrame({
    "doc_id": doc_ids,
    "text": texts
})


In [11]:
print("Total passages available:", len(df))


Total passages available: 67656


Although the target was to extract 75,000 passages, the dataset subset loaded contained only 67,656 passages in total.

This occurs because:
 MS MARCO passages are distributed unevenly across queries.
 Some queries contain fewer passages than others.

Since the task allows flexibility in dataset size the extracted corpus size is still sufficient and valid for building and demonstrating a semantic search engine.


In [12]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

This line loads the pre-trained all-MiniLM-L6-v2 SentenceTransformer model, which converts text passages into 384-dimensional dense embeddings that capture semantic meaning, allowing us to compare queries and documents using vector similarity for efficient and accurate semantic search.

In [13]:
import torch
print(torch.cuda.is_available())


True


This code checks whether a CUDA-enabled GPU is available in the current runtime.  
The torch.cuda.is_available function returns True if PyTorch can access a GPU, allowing computations to be accelerated using CUDA otherwise, it returns False, meaning all operations will run on the CPU.


In [14]:
embeddings = model.encode(
corpus_df["text"].tolist(),
batch_size=64,
show_progress_bar=True,
convert_to_numpy=True
)


print("Embedding shape:", embeddings.shape)

Batches:   0%|          | 0/1058 [00:00<?, ?it/s]

Embedding shape: (67656, 384)


This code turns all texts into numerical embeddings and stores them as a NumPy array. batch_size speeds it up, show_progress_bar shows progress and embeddings.shape tells the number of texts and embedding size.

In [15]:
faiss.normalize_L2(embeddings)


index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)


print("Total vectors indexed:", index.ntotal)

Total vectors indexed: 67656


It prepares your embeddings for fast similarity search.

In [16]:
def recall_at_k(preds, gold, k=10):
    hit = 0
    for p, g in zip(preds, gold):
        if len(set(p[:k]) & g) > 0:
            hit += 1
    return hit / len(preds)


def mrr_at_k(preds, gold, k=10):
    total = 0
    for p, g in zip(preds, gold):
        for rank, doc in enumerate(p[:k], start=1):
            if doc in g:
                total += 1 / rank
                break
    return total / len(preds)


def evaluate_dense_retriever(k=10, limit=500):
    predictions = []
    gold = []

    for q, g in zip(queries[:limit], relevant_docs[:limit]):
        res = semantic_search(q, top_k=k)
        predictions.append(res["text"].tolist())
        gold.append(g)

    print("Recall@10:", recall_at_k(predictions, gold, k))
    print("MRR@10:", mrr_at_k(predictions, gold, k))


This code checks how well a semantic search model is working. It runs search for each query and compares the results with the correct documents. Recall@10 shows whether at least one correct result appears in the top 10 and MRR@10 shows how high the first correct result is ranked. Together, these scores tell us how accurate and effective the search system is.

In [17]:
def semantic_search(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)

    scores, indices = index.search(query_embedding, top_k)

    results = corpus_df.iloc[indices[0]].copy()
    results["score"] = scores[0]

    return results


This function takes a query, converts it into an embedding and normalizes it. It then searches the FAISS index for the top_k most similar texts, retrieves them from the DataFrame and adds their similarity scores, and returns the results.

In [18]:
query = "tell me about ieee"
results = semantic_search(query, top_k=5)
results

,doc_id,text,score
50942,50942,The IEEE (Institute of Electrical and Electron...,0.795943
50943,50943,IEEE is the acronym for Institute of Electrica...,0.759019
50947,50947,The Institute of Electrical and Electronics En...,0.754995
50949,50949,IEEE stands for Institute of Electrical and El...,0.738919
50945,50945,IEEE stands for the Institute of Electrical an...,0.735959


This output shows the result of a semantic search performed with a query "tell me about ieee".  
The table lists the top 5 most semantically similar passages, where doc_id identifies the document, text contains the retrieved passage.
Higher scores indicate stronger semantic relevance, which is why the top results accurately describe IEEE.


In [19]:
semantic_search("machine learning in healthcare")


,doc_id,text,score
59195,59195,"This discussion document, produced by the Heal...",0.498215
59199,59199,Safe and efficient patient care relies on high...,0.448243
57673,57673,"Machine learning is grounded in hard sciences,...",0.435717
59198,59198,IMPROVING DATA QUALITY: A GUIDE FOR DEVELOPING...,0.423649
65377,65377,A company s medical program is an important pa...,0.419382


This line performs a semantic search for the query machine learning in healthcare.

In [20]:
query = "Tell me about Electrical engineering"
results = semantic_search(query, top_k=10)

for i, row in results.iterrows():
    print(f"\nScore: {row['score']:.4f}")
    print(row['text'][:300], "...")



Score: 0.6497
During their electricians training they will need to become familiar with electrical circuity theory. Luckily, electrical circuity theory is a part of most high school Physics courses. This makes taking at least one physics course during high school an excellent idea for potential electricians. The  ...

Score: 0.6408
To become an electrical engineer, you would normally need to complete a foundation degree, HNC, HND or degree in electrical or electronic engineering or related subject, for example building services engineering. With a degree in a relevant subject, you may be able to join a company's graduate train ...

Score: 0.6179
To go to University you need to pass 4 subjects reasonably well and these must include Mathematics, Geometry, Science and a language. Universities and Universities of Technology require Mathematics and Physical Science as compulsory subjects for entry into their Electrical Engineering courses. In yo ...

Score: 0.6160
In your grade 9 year you 

This version of the code differs in output format because the results are printed manually using a loop instead of being displayed as a DataFrame.  
Rather than showing all columns  in tabular form, it prints each result one by one with its similarity score and only the first 300 characters of the passage.  
This makes the output more readable and suitable for quick inspection.


In [21]:
semantic_search("tell me about the first prime minister of india", top_k=10)

,doc_id,text,score
24841,24841,Dear friend the first Prime Minister of India ...,0.805561
24842,24842,Pandit Jawaharlal Nehru (14 November 1889 – 27...,0.680309
24837,24837,Pandit Jawaharlal Nehru was the first and long...,0.660930
24838,24838,"Since 1947, India has had fifteen prime minist...",0.650321
24835,24835,jawaharlal nehru Jawaharlal Nehru (Hindi: जवाह...,0.645070
24839,24839,The first prime minister of India who didn’t b...,0.637164
24836,24836,Since independence India has seen 14 full-time...,0.634638
13516,13516,Government. India is a federal state with a pa...,0.524394
24840,24840,Indira Gandhi was the first woman Prime Minist...,0.513253
13518,13518,"India, officially the Republic of India (Bhāra...",0.497262


In [22]:
!pip install -q rank-bm25


This command installs the rank-bm25 library which is required for the bonus part of the evaluation criteria.

In [23]:
print("\n--- Dense Retriever Evaluation ---")
evaluate_dense_retriever()



--- Dense Retriever Evaluation ---
Recall@10: 0.898
MRR@10: 0.48859682539682536


The function evaluate_dense_retriever() is then called to measure the performance of a dense retrieval model, such as how well it finds relevant documents for given queries.

In [24]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split() for doc in corpus_df["text"]]
bm25 = BM25Okapi(tokenized_corpus)


In [25]:
def bm25_search(query, top_k=10):
    scores = bm25.get_scores(query.split())
    top_idx = np.argsort(scores)[::-1][:top_k]
    return corpus_df.iloc[top_idx]


This code sets up BM25 document ranking for text search.
Each document is tokenized by splitting text into words.
The bm25_search function then takes a query, calculates relevance scores for all documents, sorts them in descending order and returns the top-k most relevant documents from the corpus.

In [26]:
bm25_search("machine learning in healthcare")

,doc_id,text
57668,57668,The Data Analytics Scientist will work in team...
57673,57673,"Machine learning is grounded in hard sciences,..."
30554,30554,In 1877 the world's first crochet machine was ...
10248,10248,He patented the first practical sewing machine...
30558,30558,"In 1790, the English inventor Thomas Saint inv..."
50722,50722,Vicarious Learning/Observational Learning/Soci...
30555,30555,"by Tomo · March 23, 2011. A German named Charl..."
30556,30556,He patented the first practical sewing machine...
30560,30560,It was in 1846 that Elias Howe got the first A...
41266,41266,1 Pneumatic tattoo machine: Tattoo artist Cars...


Semantic search looks at the meaning of the query. It can find related results even if the exact words are different.

BM25 search looks for exact keywords in the text. It works well when the same words appear in the document.

In [27]:
def failure_analysis():
    q = "quantum cryptography standards"
    print("Query:", q)
    print("\nDense Results:")
    print(semantic_search(q)["text"].iloc[0][:300])

    print("\nBM25 Results:")
    print(bm25_search(q)["text"].iloc[0][:300])


In [28]:
failure_analysis()


Query: quantum cryptography standards

Dense Results:
Quantum computing studies theoretical computation systems (quantum computers) that make direct use of quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. Quantum computers are different from digital computers based on transistors. There exist quantum 

BM25 Results:
To be more specific, the field of quantum information studies the implications that quantum mechanics has on the fundamental nature of information. By studying this relationship between quantum theory and information, it is possible to design a new type of computer— a quantum computer. By learning h


Failure analysis is the systematic study of why a component, system, or process fails to perform its intended function. The main goal is to identify the root cause of failure and prevent it from happening again.